<a href="https://colab.research.google.com/github/luasampaio/EngenhariaDados/blob/main/58-pydeequ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Teste pydeequ no colab
- Ainda com erro ( Em analise)

In [1]:
# prompt: install deequ

!pip install pydeequ

In [2]:
!pip install pyspark pydeequ


In [3]:
!wget https://repo1.maven.org/maven2/com/amazon/deequ/deequ/1.2.2-spark-3.0/deequ-1.2.2-spark-3.0.jar -P /content/


--2025-07-01 08:48:51--  https://repo1.maven.org/maven2/com/amazon/deequ/deequ/1.2.2-spark-3.0/deequ-1.2.2-spark-3.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1258590 (1.2M) [application/java-archive]
Saving to: ‘/content/deequ-1.2.2-spark-3.0.jar.1’

deequ-1.2.2-spark-3 100%[===================>]   1.20M  --.-KB/s    in 0.01s   

2025-07-01 08:48:51 (126 MB/s) - ‘/content/deequ-1.2.2-spark-3.0.jar.1’ saved [1258590/1258590]



In [5]:
!pip install pyspark==3.0.3


In [8]:
# prompt: exemplo deequ

import os

# Set the SPARK_VERSION environment variable
os.environ["SPARK_VERSION"] = "3.5"

import pandas as pd
from pyspark.sql import SparkSession
from pydeequ.checks import *
from pydeequ.verification import *


spark = (
    SparkSession.builder
    .appName("PyDeequTest")
    .config("spark.jars.packages", "com.amazon.deequ:deequ:1.2.2-spark-3.0")
    .getOrCreate()
)


# Create a sample Pandas DataFrame
data = {'ID': [1, 2, 3, 4, 5],
        'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
        'Age': [25, 30, 35, None, 40],
        'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']}
df = pd.DataFrame(data)

# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Define Deequ Checks
check = Check(spark, CheckLevel.Warning, "Deequ Checks") \
  .hasSize(lambda x: x >= 5) \
  .isUnique("ID") \
  .isComplete("Name") \
  .isContainedIn("City", ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Seattle"]) \
  .isGreaterThan("Age", 0)

# Run the verification suite
verificationResult = VerificationSuite(spark) \
    .onData(spark_df) \
    .addCheck(check) \
    .run()

# Print the verification result
checkResultDataFrame = VerificationResult.checkResultsAsDataFrame(spark, verificationResult)
checkResultDataFrame.show()

# Stop the Spark session
spark.stop()

TypeError: code() argument 13 must be str, not int